# Setup

In [1]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

# Path('../backend/shuscribe').resolve()
sys.path.insert(0, str(Path('../backend').resolve()))

In [2]:
# Import necessary modules
import asyncio
from dotenv import load_dotenv
import os
from shuscribe.services.llm.session import LLMSession
from shuscribe.services.llm.providers.provider import (
    Message, GenerationConfig
)
from shuscribe.schemas.llm import MessageRole

load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
ANTHROPIC_API_KEY = os.environ["ANTHROPIC_API_KEY"]
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]

TEST_MODELS ={
    "openai": "gpt-4o-mini",
    "anthropic": "claude-3-5-haiku-20241022",
    "gemini": "gemini-2.0-flash-001"
}

TEST_THINKING_MODELS = {
    "openai": "o3-mini-2025-01-31",
    "anthropic": "claude-3-7-sonnet-20250219",
    "gemini": "gemini-2.0-flash-thinking-exp"
}

# Helper function to run async code in notebook
async def run_async(coro):
    return await coro

In [8]:
# Streaming response
from typing import Optional
from shuscribe.services.llm.streaming import StreamChunk, StreamStatus


async def stream(provider_name: str, messages: list[Message]):
    async with LLMSession.session_scope() as session:
        # Create a streaming config
        config = GenerationConfig(temperature=0.7)
        
        print(f"{TEST_MODELS[provider_name]}:")
        
        # last_chunk: Optional[StreamChunk] = None

        async for chunk in session.generate_stream(
            messages=messages,
            provider_name=provider_name,
            model=TEST_MODELS[provider_name],
            config=config
        ):
            print(chunk.text, end="", flush=True)

    if chunk:
        if chunk.status == StreamStatus.COMPLETE:
            return chunk.accumulated_text


In [9]:
import yaml

from shuscribe.schemas.pipeline import Chapter, StoryMetadata

with open("../backend/tests/resources/pokemon_amber/_meta.yaml", "r") as f:
    meta = yaml.safe_load(f)
    STORY_METADATA = StoryMetadata(
        title=meta.get('story_title'),
        description=meta.get('story_description'),
        genres=meta.get('genres'),
        additional_tags=meta.get('additional_tags')
    )

CHAPTERS = []
for chapter in meta.get('chapters'):
    with open(f"../backend/tests/resources/pokemon_amber/{chapter}", "r") as f:
        chapter_id = chapter.split('.')[0]
        try:
            chapter_content = yaml.safe_load(f)
            CHAPTERS.append(Chapter(id=chapter_id, title=chapter_content.get('title'), content=chapter_content.get('content')))
        except Exception:
            continue


# Chapter Summary

In [10]:
# Set Provider Name
PROVIDER_NAME = "gemini"
CHAPTER_INDEX = 0

In [ ]:
from shuscribe.services.llm.prompts.manager import PromptManager

prompt_manager = PromptManager()

messages: list[Message] = prompt_manager.chapter.summary( 
    current_chapter=CHAPTERS[CHAPTER_INDEX],
    story_metadata=STORY_METADATA,
    # current_chapter=Chapter(id=1, title="Chapter 1", content="This is a test chapter.")
)

# print(messages[-1].content)

streaming_response = await run_async(stream(PROVIDER_NAME, messages))

gemini-2.0-flash-001:
<|STARTOFSUMMARY|>
# Chapter Summary

*   The chapter is a test chapter.


Database module not implemented. Skipping save.
